##### duration: nearly instant

In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
from pathlib import Path
import sys
import logging
logging.basicConfig(level=logging.INFO)
print(sys.executable)

/n/groups/datta/tim_sainburg/conda_envs/peromoseq/bin/python3


### Get recording info (google sheets)

In [43]:
import requests
import pandas as pd
from io import BytesIO

In [44]:
# spreadsheet_url = 'https://docs.google.com/spreadsheet/ccc?key=14HIqUaSl_n-91hpAvmACY_iVY9nLKdlA6qklhxfZon0&output=csv&gid=0'
spreadsheet_url = "https://docs.google.com/spreadsheet/ccc?key=1jACsUmxuJ9Une59qmvzZGc1qXezKhKzD1zho2sEfcrU&output=csv&gid=0"
response = requests.get(spreadsheet_url)
recording_df = pd.read_csv(BytesIO(response.content))

In [45]:
recording_df[:3]

,Subject,duration_m,video_recording_id,ephys_id,calibration_id,calibration_board_shape,calibration_square_size,location_on_o2,ephys_location_on_o2
0,M04002,10,24-05-01-13-26-43-110846,2024-05-01_13-26-37,24-05-01-13-45-07-825493,NaN,NaN,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...


In [46]:
recording_df.iloc[0].location_on_o2

'/n/groups/datta/tim_sainburg/datasets/chronic24/video/M04002/24-05-01-13-26-43-110846'

### Run sync

In [47]:
for idx, recording_row in recording_df.iterrows():
    break

In [48]:
CAMERA_SAMPLERATE = 150
TRIGGER_PIN = 2
NPX_SAMPLERATE = 30000

In [49]:
output_directory = Path("/n/groups/datta/tim_sainburg/datasets/scratch/") / "240806-3d-pipeline"

In [50]:
output_directory_openephys_sync = output_directory / 'openephys_sync' / recording_row.ephys_id
output_directory_openephys_sync.mkdir(parents=True, exist_ok=True)

In [51]:
from multicamera_airflow_pipeline.tim_240731.sync.sync_cameras_to_openephys import OpenEphysSynchronizer

In [52]:
synchronizer = OpenEphysSynchronizer(
    camera_sync_file = output_directory  / 'camera_sync' / 'camera_sync.csv',
    ephys_recording_path = Path(recording_row.ephys_location_on_o2) / recording_row.ephys_id,
    ephys_sync_output_directory = output_directory_openephys_sync,
    NPX_SAMPLERATE = NPX_SAMPLERATE,
    CAMERA_SAMPLERATE = CAMERA_SAMPLERATE,
)

In [53]:
synchronizer.run()

INFO:multicamera_airflow_pipeline.tim_240731.sync.sync_cameras_to_openephys:Syncing Record Node 107#Neuropix-PXI-100.ProbeA-AP


  0%|          | 0/44953 [00:00<?, ?it/s]

  0%|          | 0/44953 [00:00<?, ?it/s]

doing correlation:   0%|          | 0/46479 [00:00<?, ?it/s]

INFO:multicamera_airflow_pipeline.tim_240731.sync.sync_cameras_to_openephys:	 creating ephys alignment mmap
INFO:multicamera_airflow_pipeline.tim_240731.sync.sync_cameras_to_openephys:Syncing Record Node 107#Neuropix-PXI-100.ProbeA-LFP


In [54]:
!ls {output_directory_openephys_sync}

Record Node 107#Neuropix-PXI-100.ProbeA-AP
Record Node 107#Neuropix-PXI-100.ProbeA-LFP


In [55]:
!ls "/n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/openephys_sync/Record Node 107#Neuropix-PXI-100.ProbeA-AP"

alignment.jpg  correlation.jpg	ephys_alignment.mmap
